### Gradio and HuggingFace

In this demo, we show how to build ready to deploy or use deep learning models. 

Huggingface hosts thousands of pre-trained models. They also built high-level APIs so we can easily use and deploy pre-trained models. 

`gradio` provides APIs so we can easily build web applications that use our pre-trained models from Huggingface. `gradio` also provides APIs so we can easily incorporate input and output web UIs.

After building the `gradio`, we can have permanent hosting using Huggingface Spaces. 

Let us first install Huggingface `transformers` and `gradio`.

**Note:** For some examples, it is best to launch the app in another tab to enable access to the required inputs such as microphone or webcam. Running the app may also lock the `python` kernel and the notebook becomes unresponsive. In that case, please restart the kernel and clear the output. 

In [ ]:
!pip install transformers
!pip install gradio

#### Hello world in gradio

As a tradition, let us build the simplest `gradio` app. It accepts a `text` input and calls the `greet()` function to process this input.

To finally see our application, we call `launch()` after constructing our `gradio` `Interface`.

In [1]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!!"

gr.Interface(fn=greet, inputs="text", outputs="text").launch()

Running on local URL:  http://127.0.0.1:7863/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x7fe2b0b0ea00>,
 'http://127.0.0.1:7863/',
 None)

#### Object Recognition using ResNet18

In our discussion about PyTorch, we used a pre-trained ResNet18 model from `torchvision`. We use `jupyter` notebook to show the results. The `jupyter` notebook is not an application that we can deploy and other people use with ease. The same with Google's colab. 

In this example, we use `gradio` to build a simple app that an end user can easily interact with. We reuse the code from our previous [example](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/pytorch_demo.ipynb).

In [2]:
import gradio as gr
import torch 
import torchvision
import torchvision.transforms as transforms
import requests
from einops import rearrange

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

resnet = torchvision.models.resnet18(pretrained=True)
resnet.eval()

# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

def classify(img):
    # By default, gradio image is numpy
    img = torch.from_numpy(img)
    # Numpy image is channel last. PyTorch is channel 1st.
    img = rearrange(img, 'h w c -> c h w')
    
    # The transforms before prediction
    img = torchvision.transforms.Resize(256)(img)
    img = torchvision.transforms.CenterCrop(224)(img).float()/255.
    img = normalize(img)
    
    # We insert batch size of 1
    img = rearrange(img, 'c h w  -> 1 c h w')
    
    # The actual prediction
    with torch.no_grad():
        pred = resnet(img)
    
    # Convert the prediction to probabilities
    pred = torch.nn.functional.softmax(pred, dim=1)
    # Remove the batch dim. torch.squeeze() can also be used.
    pred = rearrange(pred, "1 j->j")
    
    # torch to numpy space
    pred = pred.cpu().numpy()
    
    return {labels[i]: float(pred[i]) for i in range(1000)}
    
# Input to ResNet18
image = gr.inputs.Image(shape=(224, 224))
# We print top5 classes
label = gr.outputs.Label(num_top_classes=5)
examples = ['data/wonder_cat.jpg', 'data/aki_dog.jpg',]

gr.Interface(fn=classify, 
             inputs=image, 
             outputs=label, 
             title="1k Object Recognition",
             examples = ['wonder_cat.jpg', 'aki_dog.jpg',],
             description="Demonstrates how to use a pre-trained model from torchvision for image classification.",
             ).launch()

Running on local URL:  http://127.0.0.1:7864/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x7fe2b0b0ea00>,
 'http://127.0.0.1:7864/',
 None)

#### Using HuggingFace and Gradio

Loading a pre-trained model from torchvision, pre-processing the input, and post processing the output are all messy. Sometimes, we just want to load and use a machine learning model. Huggingface provides a shortcut for all these steps through the use of `pipeline`. In `pipeline`, we supply the task name and the pre-trained model that is stored in Huggingface Model Hub.

In this example, we use a much better model compared to ResNet18. It is called [BEIT](https://arxiv.org/abs/2106.08254) and can classify objects up to about 22k categories. We construct the `gradio` app by calling `from_pipeline()`.

In [3]:
import gradio as gr
from transformers import pipeline

pipe = pipeline(task="image-classification", 
                 # model that can do 22k-category classification
                 model="microsoft/beit-base-patch16-224-pt22k-ft22k")
gr.Interface.from_pipeline(pipe, 
                           title="22k Image Classification",
                           description="Object Recognition using Microsoft BEIT (22k classes)",
                           examples = ['wonder_cat.jpg', 'aki_dog.jpg',],
                           ).launch()

/Users/rowel/anaconda3/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Running on local URL:  http://127.0.0.1:7865/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x7fe2b0b0ea00>,
 'http://127.0.0.1:7865/',
 None)

#### Automatic Speech Recognition (ASR)

Let us shift to audio or speech domain. In this example, we demonstrate an Automatic Speech Recognition (ASR) system. We will use our microphone to record audio which is then converted to text using ASR. In this example, best to open the application in another browser tab by setting `inbrowser=True`.

Before running the `gradio` app, this ASR requires `sentencepice` module. Let us install it first.

In [ ]:
!pip install sentencepiece

In this ASR, we use Huggingface pipeline.

In [4]:
import gradio as gr
from transformers import pipeline

model = pipeline(task="automatic-speech-recognition", 
                 model="facebook/s2t-medium-librispeech-asr")
gr.Interface.from_pipeline(model,
                           title="Automatic Speech Recognition (ASR)",
                           description="Using pipeline with Facebook S2T for ASR.",
                           examples=['data/ljspeech.wav',],
                           ).launch(inbrowser=True)

Running on local URL:  http://127.0.0.1:7866/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x7fe2b0b0ea00>,
 'http://127.0.0.1:7866/',
 None)

/Users/rowel/anaconda3/lib/python3.8/site-packages/transformers/models/speech_to_text/modeling_speech_to_text.py:566: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  input_lengths = (input_lengths - 1) // 2 + 1
/Users/rowel/anaconda3/lib/python3.8/site-packages/transformers/models/speech_to_text/modeling_speech_to_text.py:566: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch

#### Text to Speech (TTS)

Let us do the reverse of ASR or Text to Speech (TTS). In this example, we supply text and this text is converted to speech using the voice of Linda Johnson. We use a pre-trained model of [FastSpeech2](https://arxiv.org/abs/2006.04558) that is provided by Facebook in Model Hub.

In this example, we use [`load()`](https://gradio.app/docs/#load) method to load the pre-trained model. 

In [5]:
import gradio as gr
gr.Interface.load("huggingface/facebook/fastspeech2-en-ljspeech", 
                  description="TTS using FastSpeech2",
                  title="Text to Speech (TTS)",
                  examples=[["The quick brown fox jumps over the lazy dog."]]
                  ).launch()

Fetching model from: https://huggingface.co/facebook/fastspeech2-en-ljspeech
Running on local URL:  http://127.0.0.1:7867/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x7fe2b0b0ea00>,
 'http://127.0.0.1:7867/',
 None)

#### Text Generation using GPT2

In this example, we use a large language model (LLM) by OpenAI to generate text. It is called [GPT2](https://github.com/openai/gpt-2). Text generation is one of the tasks where a language model can help us. Basically, we provide the initial text made of few words. Then, the model will continue it.

In [7]:
import gradio as gr
gr.Interface.load("huggingface/gpt2",
                  title="Text Generation",
                  description="Using GPT2.",
                  ).launch(inbrowser=True)

Fetching model from: https://huggingface.co/gpt2
Running on local URL:  http://127.0.0.1:7868/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x7fe2b0b0ea00>,
 'http://127.0.0.1:7868/',
 None)

#### Using `gr.Series()` to automatically chain the I/O of multiple models

In this example, we use the previous text generator as input to our TTS. We can for example use to generate a podcast on a certain topic.

In [ ]:
import gradio as gr


textgen = gr.Interface.load("huggingface/gpt2",
                            title="Text Generation",
                            description="Using GPT2.",
                            )

tts = gr.Interface.load("huggingface/facebook/fastspeech2-en-ljspeech", 
                        description="TTS using FastSpeech2",
                        title="Text to Speech (TTS)",
                        examples=[["The quick brown fox jumps over the lazy dog."]]
                        )

gr.Series(textgen, tts).launch(show_error=True, debug=True)


Fetching model from: https://huggingface.co/gpt2
Fetching model from: https://huggingface.co/facebook/fastspeech2-en-ljspeech
Running on local URL:  http://127.0.0.1:7869/

To create a public link, set `share=True` in `launch()`.
